In [64]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from shapely.wkt import loads

In [85]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コード : {finish_pref}')





#座標変換
def crs_trf(gdf, dst_proj):
  if gdf.crs is None:
      gdf.set_crs(epsg=dst_proj, inplace=True)
  else:
      gdf.to_crs(epsg=dst_proj, inplace=True)
 
 
      
      

#浸水深スケーリング
def flood_rank_scaling(column):
    flood_rank = np.where((column == 11) | (column == 21), 1, 
                    np.where((column == 12) | (column == 22), 2, 
                        np.where((column == 13) | (column == 23), 3,
                            np.where((column == 14) | (column == 24) | (column == 25) | (column == 26), 4, 5))))
    return flood_rank


     


#最大―最小スケーリング
def min_max_scaling_numpy(column):
    #指定のカラムをnumpy配列に
    array = column.to_numpy()
    
    #最大―最小スケーリングを実施
    min_val = np.min(array)
    max_val = np.max(array)
    scaled_array = (array - min_val) / (max_val - min_val)
    return scaled_array






def subsidy(burden_df):
    #補助金とそれに対応する移転希望者の割合の辞書を作成
    subsidy_dic = {0.1:0.05,
                   0.2:0.07,
                   0.3:0.08,
                   0.4:0.1,
                   0.5:0.12,
                   0.6:0.14,
                   0.7:0.17,
                   0.8:0.18,
                   0.9:0.19,
                   1.0:0.21}

    # 総和を保存するための空のデータフレームを作成
    subsidy_df = pd.DataFrame(columns=['subsidy_ratio', 'transfer_ratio', 'total_subsidy', 'measures_damage_cost','measures_total_cost', 'without_measures_cost'])

    #補助金毎にループ
    for col, frac in subsidy_dic.items():
        #burden_dfにdamage_priceに対する補助する金額の列を追加
        burden_df[col] = burden_df['house_price']*col


        # データフレームから指定された割合で行をランダムに選ぶ
        sample_df = burden_df[['house_price', 'damage_price', col]].sample(frac=frac)

        # 選択された行に含まれる数値の総和を計算
        total_sum = sample_df[col].sum()
        
        
        #選択された住宅の移転費用と損害額を比較する必要がある．いらない変数といる変数を分ける．
        
        #補助金と損害額の総和
        
        #施策なしの損害額
        without_measures_cost = burden_df['damage_price'].sum()
        #施策有の損害額
        measures_damage_cost = without_measures_cost - sample_df['damage_price'].sum()
        #補助金と損害額の総和
        measures_total_cost = measures_damage_cost + total_sum
        

        # 総和を一時的なデータフレームに保存
        temp_df = pd.DataFrame({
            'subsidy_ratio': [f"subsidy_{col * 100}%"],
            'transfer_ratio': [f"transfer_{frac * 100}%"],
            'total_subsidy': [total_sum],
            'measures_damage_cost':[measures_damage_cost],
            'measures_total_cost':[measures_total_cost],
            'without_measures_cost':[without_measures_cost]
            })

        # 結果をメインのデータフレームに追加
        subsidy_df = pd.concat([subsidy_df, temp_df], ignore_index=True)
        

    return subsidy_df

    










%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
pref = int(start_pref)
while pref <= int(finish_pref):
    
    
    
    
    
    #市区町村ポリゴンを取得
    total_city_polygon_path = './data/市区町村ポリゴン/japan_ver84/japan_ver84.shp'
    total_city_polygon = gpd.read_file(total_city_polygon_path)
    city_polygon =  total_city_polygon[total_city_polygon["JCODE"].str[:2] == f'{pref:02}']
    
    # NaNを空文字列で置き換える
    city_polygon['SICHO'].fillna('', inplace=True)
    city_polygon['GUN'].fillna('', inplace=True)
    city_polygon['SEIREI'].fillna('', inplace=True)
    city_polygon['SIKUCHOSON'].fillna('', inplace=True)
    #群，政令市と市区町村名を結合
    city_polygon['city'] = city_polygon['SICHO'] + city_polygon['GUN'] + city_polygon['SEIREI']+ city_polygon['SIKUCHOSON']
    
    
    
    
    
    
    #地価価格の読み込み
    land_price_path = f'./data/土地情報センター/{pref}.csv'
    land_price = pd.read_csv(land_price_path, encoding='shift-jis')

    
    
    
    
    
    #建物の損害額
    burden_path = f'./dev/建物家財損害額/{pref}_lost.csv'
    burden_df = pd.read_csv(burden_path, encoding='shift-jis')
    burden_df['geometry'] = loads(burden_df['geometry'])
    burden_df = gpd.GeoDataFrame(burden_df, geometry='geometry', crs=4326)
    
    
    
    
    
    
    
    
    
    
    
    #市区町村ごとに集計
    for index, row in city_polygon.iterrows():
        row_df = pd.DataFrame([row.values], columns=row.index)
        row_df = gpd.GeoDataFrame(row_df, geometry='geometry', crs=4326)
        city_bldg = gpd.sjoin(burden_df, row_df[['JCODE', 'geometry']], op='contains')

        
        
        
        
        #該当の市区町村価格を選択
        
        #土地価格から市区町村ポリゴンの市区町村名と等しい土地価格を取得する
        city_name = row_df['city'].iloc[0]
        #該当の市区町村の土地価格がdfで出力されるので['住宅地']の数値を取得するためにiatを使用
        city_land_price = land_price[land_price['地区名'] == city_name].iat[0, 1]
        #土地価格が文字列なのでカンマと型の変更を行う
        city_land_price = city_land_price.replace(',', '')
        city_land_price = int(city_land_price)
        
        #建物データに土地価格のカラムを追加
        city_bldg['land_price'] = city_bldg['AREA'] * city_land_price
        
        
        
        
        
        
        #浸水ランクが5段階と7段階があるので5段かにスケーリングする
        city_bldg['flood_rank'] = flood_rank_scaling(city_bldg['A31_001'])
        
        
        
        
        
        #最大―最小スケーリング
        city_bldg['estimated_age_scaled'] = min_max_scaling_numpy(city_bldg['estimated_age'])
        city_bldg['flood_rank_scaled'] = min_max_scaling_numpy(city_bldg['flood_rank'])
    

        
        
        
        
        #補助金の算出
        subsidy(burden_df)
        
        
        
        
        
        # ディレクトリのパスを構築
        directory_path = f'./data/新しいフォルダー/{pref}'
        # ディレクトリが存在しない場合に作成
        os.makedirs(directory_path, exist_ok=True)

        # ファイルパスを構築
        city_bldg_path = f'{directory_path}/{row['JCODE']}_building.csv'
        # CSVとして保存
        city_bldg.to_csv(city_bldg_path, index=False)
    
    
    
    
    
    pref += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 33
終了する都道府県コードを入力してください
終了する都道府県コード : 33
G:\マイドライブ\akiyamalab\卒論\負担率総集編


C:\Users\imai_14\AppData\Local\Temp\ipykernel_5100\1137686510.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_polygon['SICHO'].fillna('', inplace=True)
C:\Users\imai_14\AppData\Local\Temp\ipykernel_5100\1137686510.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_polygon['GUN'].fillna('', inplace=True)
C:\Users\imai_14\AppData\Local\Temp\ipykernel_5100\1137686510.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

ValueError: Can only compare identically-labeled Series objects

In [116]:
burden_df

,tatemon_id,x,y,geometry,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,...,age,dmg_rate,house_price,house_damage_price,assets,children,adult,add_assets,assets_damage_price,total_lost
0,1573,133.797213,34.672247,POINT (133.79721 34.67225),0,166.805,1364,166.805,331012270,1,...,25.5,0.60,4.270208e+07,4.452973e+06,11500000,2,2,4200000,9420000.0,1.387297e+07
1,1688,133.797788,34.673460,POINT (133.79779 34.67346),0,184.710,1364,184.710,331012270,1,...,15.5,0.40,4.728576e+07,9.415541e+06,11500000,2,2,4200000,6280000.0,1.569554e+07
2,1694,133.797064,34.672907,POINT (133.79706 34.67291),0,102.940,1364,102.940,331012270,1,...,5.0,0.40,2.635264e+07,8.833405e+06,11500000,2,4,5800000,6920000.0,1.575340e+07
3,1701,133.797125,34.672804,POINT (133.79712 34.67280),0,68.600,1364,68.600,331012270,1,...,15.5,0.40,1.756160e+07,3.496866e+06,3000000,0,5,4000000,2800000.0,6.296866e+06
4,1703,133.796789,34.672734,POINT (133.79679 34.67273),0,115.070,1364,115.070,331012270,1,...,5.0,0.40,2.945792e+07,9.874295e+06,3000000,0,3,2400000,2160000.0,1.203429e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231777,19890,133.816148,34.860425,POINT (133.81615 34.86043),0,107.045,1364,107.045,33681059005,1,...,15.5,0.40,2.740352e+07,5.456589e+06,5000000,4,2,6800000,4720000.0,1.017659e+07
231778,19898,133.817725,34.860710,POINT (133.81772 34.86071),0,118.770,1364,118.770,33681059006,1,...,5.0,0.40,3.040512e+07,1.019180e+07,11500000,2,2,4200000,6280000.0,1.647180e+07
231779,19908,133.816064,34.861105,POINT (133.81606 34.86110),0,168.965,1364,168.965,33681059006,1,...,8.0,0.40,4.325504e+07,1.281733e+07,11500000,2,2,4200000,6280000.0,1.909733e+07
231780,19909,133.816188,34.860946,POINT (133.81619 34.86095),0,181.605,1364,181.605,33681059006,1,...,15.5,0.40,4.649088e+07,9.257264e+06,11500000,2,2,4200000,6280000.0,1.553726e+07


In [113]:

city_name = row_df['city'].iloc[0]
city_land_price = land_price[land_price['地区名'] == city_name]

In [114]:
city_land_price

,地区名,住宅地,宅地見込地,商業地,工業地,全用途
2,岡山市北区,"82,500",NaN,"265,700","47,800","167,700"


In [90]:
row_df['city']

0    岡山市北区
Name: city, dtype: object

In [91]:
land_price['地区名']

0          岡山県
1          岡山市
2        岡山市北区
3        岡山市中区
4        岡山市東区
5        岡山市南区
6          倉敷市
7          津山市
8          玉野市
9          笠岡市
10         井原市
11         総社市
12         高梁市
13         新見市
14         備前市
15         赤磐市
16         真庭市
17         美作市
18         浅口市
19      和気郡和気町
20      都窪郡早島町
21      浅口郡里庄町
22      小田郡矢掛町
23      苫田郡鏡野町
24      勝田郡勝央町
25    加賀郡吉備中央町
Name: 地区名, dtype: object